# <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">Santader Customer Transaction Classification.</p>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Another_new_Santander_bank_-_geograph.org.uk_-_1710962.jpg/640px-Another_new_Santander_bank_-_geograph.org.uk_-_1710962.jpg"  width="1000" height="600">

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Import Modules**</p>


In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold
import warnings
from six.moves import urllib
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn')
from scipy.stats import norm, skew
from sklearn.decomposition import PCA
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import math
import plotly.express as px
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
import eli5
from eli5.sklearn import PermutationImportance
import shap
from scipy import stats
from sklearn.metrics import roc_curve, auc
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from scipy.stats import skew
from sklearn.model_selection import GridSearchCV


## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Loading The Data**</p>


In [ ]:
#Load the Data
train = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv')
test = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
train.head()

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Basic Analysis**</p>


In [ ]:
train.shape, test.shape

In [ ]:
print('Column/Columns that are not in test data: ', end = '')
for i in train.columns:
    if i not in test.columns:
        print(i)

> **Test data do not have target column.**

In [ ]:
train.describe()

> **From initial analysis, I do not see max values to be way too out of bounds for some of the columns, so I don't think there are going to be many outliers. However, I would use an IQR to check for outliers as there are 200 columns and it would be difficult to investigate like this.** 

> **Also, I can see skewness exists in almost every variable. We are going to check further using displots.**

> **Dropping ID_code, won't be of any use.**

In [ ]:
train.drop('ID_code', axis = 1, inplace = True)

In [ ]:
train[train.duplicated()]

**No duplicates exist**

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Checking For The Null Values** </p>


In [ ]:
train.isnull().sum().values

> **All 0's exist. No null values.**

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Checking For Class Imbalance**</p>


In [ ]:

sns.countplot(train['target']);

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Checking For Skewness**</p>


In [ ]:
right_skewed, left_skewed, no_skeweness = 0, 0, 0

for col in list(set(train.columns) - set(['target'])):
    if skew(train[col])>0:
        right_skewed+=1        
    elif skew(train[col])==0:
        no_skeweness+=1
    else:
        left_skewed+=1

print("Features that are left skewed are: ", left_skewed)
print("Features that are left skewed are: ", right_skewed)
print("Features that are not skewed are: ", no_skeweness)

#### <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Visualizing Right Skewness in Var_77 And How It Looks After Transformation**</p>


In [ ]:
fig, axes = plt.subplots(1, 2);
plt.figure(figsize = (10,8));
sns.distplot(train['var_77'], ax=axes[0]);
axes[0].set_title("Original distribution showing right skewness");
sns.distplot(np.sqrt(train['var_77']), ax=axes[1]);
axes[1].set_title('After applying the transformation');

#### <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Visualizing Left Skewness in Var_78 And How It Looks After Transformation**</p>


In [ ]:
 
fig, axes = plt.subplots(1, 2);
plt.figure(figsize = (10,8));
sns.distplot(train['var_78'], ax=axes[0]);
axes[0].set_title("Original distribution showing left skewness");
sns.distplot(np.power(train['var_78'], 1.5), ax=axes[1]);
axes[1].set_title('After applying the transformation');

#### <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Checking For Outliers**</p>


In [ ]:
fig = px.box(train, y="var_0",
      title="Boxplot for var_0")
fig.update_layout(

    title_font_color="#fff",paper_bgcolor="#000000",title_font_size=20,title_x=.5,font_color="#bbb",
    plot_bgcolor="#D0D3D4")

fig.update_yaxes(showgrid=False,visible=False,title="percent of comments")
fig.update_xaxes(tickfont_size=9)
fig.show()

> **Outliers exists.**

> **Outliers have been removed now.**

> **Removed datapoints that fall outside  3 s.d**

In [ ]:
train = train[(np.abs(stats.zscore(train)) < 3).all(axis=1)]
print("New shape after removing outliers:", train.shape)

In [ ]:
X, y = train.drop('target', axis = 1), train ['target']

In [ ]:
len(train[train['target'] == 0])/ len(train[train['target'] == 1])
    


### **Key points:**

> **There's a pretty good imbalance.**

> **For every sample of class 1, 9.03 samples of class 0 are present. We need to deal with class imbalance here.**

> **I would probably be using tomek links or SMOTE for dealing with it.**



## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Train Test Split** </p>


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X,y 
                                                    , test_size = 0.20, random_state = 0) 
print("Shape of train data: ", X_train.shape)
print("Shape of test data: ", X_test.shape)

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Creating a Baseline Model With LGBM**</p>


In [ ]:

lgbm = LGBMClassifier(n_jobs=-1)

lgbm = lgbm.fit(X_train, y_train,eval_metric='auc',eval_set=(X_test , y_test),verbose=50,early_stopping_rounds= 50)



In [ ]:
print( 'Train roc auc score for LGBM: ', roc_auc_score(y_train, lgbm.predict_proba(X_train)[:, 1]))
print( 'Test roc auc score for LGBM: ', roc_auc_score(y_test, lgbm.predict_proba(X_test)[:, 1]))

In [ ]:
print(classification_report(y_test, lgbm.predict(X_test)))

In [ ]:
fpr, tpr, _ = roc_curve(y_test, lgbm.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='red', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="upper left")
plt.show()


## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Interpreting Baseline Model Using Shap**</p>


In [ ]:
shap.initjs()

X_shap=X_train.copy()


lgb_explainer = shap.TreeExplainer(lgbm)
lgb_shap_values = lgb_explainer.shap_values(X_shap)
shap.summary_plot(lgb_shap_values, X_shap, plot_type="bar")

> **Here we can see which variable is pushing towards 0 and 1.**

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Now that I have a baseline model, I'll try to improve it**</p>


## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Scaling the data**</p>


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Checking how much variance are different PCA components explaining**</p>


In [ ]:
for components in range(100, 199):
    pca = PCA(n_components = components)
    pca.fit(X_train)
    print('Explained variance for ', components, 'is: ', sum(pca.explained_variance_ratio_))

### **Key note**

> **For 160 components, explained variance goes as high as 81.86 which is pretty good. We have dropped down from 199 features.**

In [ ]:
pca = PCA(n_components = 160)
X_train_PCA = pca.fit_transform(X_train)
X_test_PCA = pca.transform(X_test)

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Balancing Data With Random Undersampling and Oversampling**</p>



In [ ]:
print("Before doing any balancing, the distribution looks like","class 0: ",len(y[y==0]),"class 1: " ,len(y[y==1]))

In [ ]:

over = RandomOverSampler(sampling_strategy=1)

print(Counter(y_train))

under = RandomUnderSampler(sampling_strategy=.15)
# fit and apply the transform
Xs, ys = under.fit_resample(X_train_PCA, y_train)
print(Counter(ys))

# fit and apply the transform
Xsmote, ysmote = over.fit_resample(Xs, ys)
print(Counter(ysmote))




> **This looks good. Class distribution is 1:1 now. Previously there was 1 : 9 ratio.**

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Fitting LGBM With PCA Data**</p>



In [ ]:

lgb_pca = LGBMClassifier(n_jobs=-1)

lgb_pca= lgb_pca.fit(Xsmote, ysmote,eval_metric='auc',eval_set=(X_test_PCA , y_test),verbose=50,early_stopping_rounds= 50)




In [ ]:
print( 'Train roc auc score for LGBM: ', roc_auc_score(ysmote, lgb_pca.predict_proba(Xsmote)[:, 1]))
print( 'Test roc auc score for LGBM: ', roc_auc_score(y_test, lgb_pca.predict_proba(X_test_PCA)[:, 1]))

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Tuning The HyperParamters Of LGBM**</p>



In [ ]:
# Doing a grid search using the parmeter grid
parameters = { 'max_depth': [5,6,7,8],
                 'learning_rate' : [0.01,0.02,0.03],
                  'num_iterations'    : [100, 200, 500]
                 }
grid_ = GridSearchCV(estimator=LGBMClassifier(), param_grid = parameters, cv = 2, n_jobs=4, verbose = 10)
grid_.fit(Xsmote, ysmote)

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params for LGBM:\n",grid_.best_estimator_)
print("\n The best score across ALL searched params for LGBM:\n",grid_.best_score_)
print("\n The best parameters across ALL searched params for LGBM:\n",grid_.best_params_)

# Getting the best estimator
lgbm_optimized_model = grid_.best_estimator_
lgbm_optimized_model.fit(Xsmote, ysmote)

In [ ]:
print( 'Train roc auc score for LGBM: ', roc_auc_score(ysmote, lgbm_optimized_model.predict_proba(Xsmote)[:, 1]))
print( 'Test roc auc score for LGBM: ', roc_auc_score(y_test, lgbm_optimized_model.predict_proba(X_test_PCA)[:, 1]))

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Using CatBoost + PCA**</p>



In [ ]:

cat_boost = CatBoostClassifier(eval_metric = 'AUC')

cat_boost = cat_boost.fit(Xsmote, ysmote,eval_set=(X_test_PCA, y_test),early_stopping_rounds=70,verbose=50)

print(' Roc auc score on train data:'.format(roc_auc_score(ysmote,cat_boost.predict_proba(Xsmote)[:, 1])))
print(' Roc auc score on test data: '.format(roc_auc_score(y_test,cat_boost.predict_proba(X_test_PCA)[:, 1])))

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Tuning The HyperParamters Of CatBoost**</p>



In [ ]:
parameters = {'depth'         : [4,5,6,7,8,9],
                 'learning_rate' : [0.01,0.02,0.03],
                  'iterations'    : [50,100, 200, 500]
                 }
grid_ = GridSearchCV(estimator=CatBoostClassifier(), param_grid = parameters, cv = 2, n_jobs=4, verbose = 10)
grid_.fit(Xsmote, ysmote)
            


In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params for CatBoost:\n",grid_.best_estimator_)
print("\n The best score across ALL searched params for CatBoost:\n",grid_.best_score_)
print("\n The best parameters across ALL searched params for CatBoost:\n",grid_.best_params_)



In [ ]:
# Getting the best estimator
catboost_optimized_model = grid_.best_estimator_
catboost_optimized_model.fit(Xsmote, ysmote)

In [ ]:
print( 'Train roc auc score for optimzed CatBoost: ', roc_auc_score(ysmote, catboost_optimized_model.predict_proba(Xsmote)[:, 1]))
print( 'Test roc auc score for optimized CatBoost: ', roc_auc_score(y_test, catboost_optimized_model.predict_proba(X_test_PCA)[:, 1]))

## <p style="background-color:#FFD580;font-family:newtimeroman;color:#000000;font-size:150%;text-align:center;border-radius:40px 40px;">**Stacking Model: LGBM + CatBoost**</p>



In [ ]:
stacked_clf = StackingClassifier(classifiers =[lgbm_optimized_model,catboost_optimized_model], meta_classifier = lgbm_optimized_model, use_probas = True, use_features_in_secondary = True)

stacked_clf.fit(Xsmote, ysmote)   


In [ ]:
stack_pred_train = stacked_clf.predict_proba(Xsmote)[:,1]
stack_pred_test = stacked_clf.predict_proba(X_test_PCA)[:,1]

roc_train_stacked = roc_auc_score(ysmote,stack_pred_train)
roc_test_stacked = roc_auc_score(y_test,stack_pred_test)

print( 'Train roc auc score for Stacked Classifier: ',roc_train_stacked)
print( 'Test roc auc score for Stacked Classifier: ',roc_test_stacked )

In [ ]:
labels = ['0', '1']
cm = confusion_matrix(y_test, stacked_clf.predict(X_test_PCA))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax) ;
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(labels);
ax.yaxis.set_ticklabels(labels);
    

In [ ]:
fpr, tpr, _ = roc_curve(y_test, stacked_clf.predict_proba(X_test_PCA)[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='red', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="upper left")
plt.show()


## **Conclusion:**

> **All models are performing almost the same.** 

> **I tried to do a PCA to deal with overfiitting but that did not help the cause.**

> **Tried a stacked model, so as to get a better test score but that was also nearly same.**